# 9章　クラス分類 III：音楽ジャンル分類

### 概要

* 音の単純な特徴量は周波数と位相が異なる正弦波の重なり
* 分類器の評価方法
  * 「混合行列」は対角線上が高い数値であればよい
  * 「ROC曲線」は、曲面積が大きくなればよい
* 分類器の改善には、無理せず適切な特徴量を利用しましょう

NameError: name '問いかけ' is not defined

# 参考文献

* [音声データ]()
* [ソースコード]（https://github.com/luispedro/BuildingMachineLearningSystemsWithPython/tree/first_edition）
* [音声言語処理特論]（http://www.slp.cs.tut.ac.jp/~kyama/Lecture/AdvSLP/slide/AdvSLP_02.pdf）
